In [1]:
# Import other modules not related to PySpark
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
import pylab 
import seaborn as sns
import scipy.stats as scipy_stats
from scipy.stats import probplot

# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline

In [2]:
# Import PySpark related modules
import pyspark
from pyspark.rdd import RDD

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import GBTRegressor, LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator

from pyspark.sql import Row, DataFrame, SparkSession, SQLContext, functions, Window
from pyspark.sql.types import DoubleType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import skewness, lit, desc, col,when, size, array_contains, row_number, \
isnan, udf, hour, array_min, array_max, countDistinct, median, collect_list, \
max, mean, min, stddev, monotonically_increasing_id  
from pyspark.sql.types import *

MAX_MEMORY = '10G'

# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()

In [19]:
filename_data = 'itinerariesClear.csv'

# Load the main data set into pyspark data frame 
df_z = spark.read.options(inferSchema='True', header='True', delimeter=',').csv(filename_data, mode='DROPMALFORMED')
print('Data frame type: ' + str(type(df_z)))

Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


In [20]:
from pyspark.sql.functions import col

In [21]:
data_subset = df_z.select(
 'startingAirport',
 'destinationAirport',
 'elapsedDays',
 col('isBasicEconomy').cast('Int').alias('isBasicEconomy'),
 col('isRefundable').cast('Int').alias('isRefundable'),
 col('isNonStop').cast('Int').alias('isNonStop'),
 'baseFare',
 'totalFare',
 col('totalTravelDistance').alias('label')
)
data_subset.show(10)

+---------------+------------------+-----------+--------------+------------+---------+--------+---------+-----+
|startingAirport|destinationAirport|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|label|
+---------------+------------------+-----------+--------------+------------+---------+--------+---------+-----+
|            LAX|               SFO|          0|             1|           0|        1|   26.98|     43.6|  339|
|            LAX|               SFO|          0|             0|           0|        1|   26.98|     43.6|  339|
|            LAX|               SFO|          0|             1|           0|        1|   26.98|     43.6|  339|
|            LAX|               SFO|          0|             1|           0|        1|   26.98|     43.6|  339|
|            LAX|               SFO|          0|             1|           0|        1|   45.58|     63.6|  339|
|            LAX|               SFO|          0|             1|           0|        1|   45.58|     63.6

In [22]:
strIdx = StringIndexer(inputCols = ['startingAirport', 'destinationAirport'], outputCols 
= ['startingAirportIdx', 'destinationAirportIdx'])
oneHotEnc = OneHotEncoder(inputCols=['startingAirportIdx', 
'destinationAirportIdx'], outputCols=['startingAirportEnc', 'destinationAirportEnc'])
catVect = VectorAssembler(inputCols = ['startingAirportEnc', 
'destinationAirportEnc', 'isBasicEconomy', 'isRefundable', 'isNonStop'], 
outputCol='catFeatures')
numVect = VectorAssembler(inputCols = ['baseFare', 'totalFare', 'elapsedDays'], 
outputCol='numFeatures')
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), 
outputCol='normFeatures')
featVect = VectorAssembler(inputCols=['catFeatures', 'normFeatures'], 
outputCol='features')
lr = LinearRegression(labelCol='label', featuresCol='features')
pipeline = Pipeline(stages=[strIdx, oneHotEnc, catVect, numVect, minMax, featVect, lr])

In [23]:
# Разделим данные на обучающую и тестовую выборки
splits = data_subset.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1].withColumnRenamed('label', 'trueLabel')
# Создаем сетку гиперпараметров
paramGrid = ParamGridBuilder() \
 .addGrid(lr.regParam, [0.0, 0.3, 0.5]) \
 .addGrid(lr.maxIter, [50, 100, 150]).build() 
# Создаем кросс-валидатор

crossval = CrossValidator(estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=RegressionEvaluator(),
    numFolds=3)
model = crossval.fit(train)

In [24]:
predictions = model.transform(test)
predictions = predictions.select('features', 'prediction', 'trueLabel')
predictions.show(50, truncate=False)


+---------------------------------------------------------------------+------------------+---------+
|features                                                             |prediction        |trueLabel|
+---------------------------------------------------------------------+------------------+---------+
|(36,[8,20,33,34],[1.0,1.0,0.0014974220518085347,0.02482154316635108])|1193.823934407215 |1579     |
|(36,[8,20,33,34],[1.0,1.0,0.0043391207183088205,0.01743238710938222])|1138.484621081247 |1579     |
|(36,[8,20,33,34],[1.0,1.0,0.02229684123355814,0.023670123566337346]) |1114.6701002239838|947      |
|(36,[8,20,33,34],[1.0,1.0,0.02229684123355814,0.023670123566337346]) |1114.6701002239838|947      |
|(36,[8,20,33,34],[1.0,1.0,0.02229684123355814,0.023670123566337346]) |1114.6701002239838|947      |
|(36,[8,20,33,34],[1.0,1.0,0.02229684123355814,0.023670123566337346]) |1114.6701002239838|947      |
|(36,[8,20,33,34],[1.0,1.0,0.02229684123355814,0.023670123566337346]) |1114.6701002239838|1

In [25]:
evaluator_mse = RegressionEvaluator(labelCol='trueLabel', 
predictionCol='prediction', metricName="mse")
mse = evaluator_mse.evaluate(predictions)
print(f'Metric "RMSE" on test data: {mse:.3f}')
evaluator_mae = RegressionEvaluator(labelCol='trueLabel', 
predictionCol='prediction', metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print(f'Metric "mae" on test data: {mae:.3f}')
evaluator_rmse = RegressionEvaluator(labelCol='trueLabel', 
predictionCol='prediction', metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print(f'Metric "rmse" on test data: {rmse:.3f}')
evaluator_r2 = RegressionEvaluator(labelCol='trueLabel', predictionCol='prediction', 
metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print(f'Metric "R^2" on test data: {r2:.3f}')


Metric "RMSE" on test data: 290714.957
Metric "mae" on test data: 408.020
Metric "rmse" on test data: 539.180
Metric "R^2" on test data: 0.556


In [26]:
# Вывод лучших параметров
best_model = model.bestModel
best_parameters = best_model.stages[-1].extractParamMap()

print("Лучшие параметры модели LinearRegression:")
for param, value in best_parameters.items():
 print(f"{param.name}: {value}")


Лучшие параметры модели LinearRegression:
aggregationDepth: 2
elasticNetParam: 0.0
epsilon: 1.35
featuresCol: features
fitIntercept: True
labelCol: label
loss: squaredError
maxBlockSizeInMB: 0.0
maxIter: 50
predictionCol: prediction
regParam: 0.0
solver: auto
standardization: True
tol: 1e-06
